This example is based on Figure 10.2 from Hastie et al 2009 [1] and illustrates the difference in performance between the discrete SAMME [2] boosting algorithm and real SAMME.R boosting algorithm. Both algorithms are evaluated on a binary classification task where the target Y is a non-linear function of 10 input features.

Discrete SAMME AdaBoost adapts based on errors in predicted class labels whereas real SAMME.R uses the predicted class probabilities.

[1]	T. Hastie, R. Tibshirani and J. Friedman, “Elements of Statistical Learning Ed. 2”, Springer, 2009.

[2]	Zhu, H. Zou, S. Rosset, T. Hastie, “Multi-class AdaBoost”, 2009.

#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

### Version

In [1]:
import sklearn
sklearn.__version__

'0.18.1'

### Imports

This tutorial imports [DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier), [zero_one_loss](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.zero_one_loss.html#sklearn.metrics.zero_one_loss) and [AdaBoostClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html#sklearn.ensemble.AdaBoostClassifier).

In [2]:
print(__doc__)

import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import zero_one_loss
from sklearn.ensemble import AdaBoostClassifier

Automatically created module for IPython interactive environment


### Calculations

In [3]:

n_estimators = 400
# A learning rate of 1. may not be optimal for both SAMME and SAMME.R
learning_rate = 1.

X, y = datasets.make_hastie_10_2(n_samples=12000, random_state=1)

X_test, y_test = X[2000:], y[2000:]
X_train, y_train = X[:2000], y[:2000]

dt_stump = DecisionTreeClassifier(max_depth=1, min_samples_leaf=1)
dt_stump.fit(X_train, y_train)
dt_stump_err = 1.0 - dt_stump.score(X_test, y_test)

dt = DecisionTreeClassifier(max_depth=9, min_samples_leaf=1)
dt.fit(X_train, y_train)
dt_err = 1.0 - dt.score(X_test, y_test)

ada_discrete = AdaBoostClassifier(
    base_estimator=dt_stump,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME")
ada_discrete.fit(X_train, y_train)

ada_real = AdaBoostClassifier(
    base_estimator=dt_stump,
    learning_rate=learning_rate,
    n_estimators=n_estimators,
    algorithm="SAMME.R")
ada_real.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          learning_rate=1.0, n_estimators=400, random_state=None)

### Plot Results

In [4]:

decision_stump = go.Scatter(x=[1, n_estimators], y=[dt_stump_err] * 2, 
                            mode='lines',
                            line=dict(color='black'),
                            name='Decision Stump Error')

decision_tree = go.Scatter(x=[1, n_estimators], y=[dt_err] * 2, 
                           mode='lines',
                           line=dict(color='black', dash='dash'),
                           name='Decision Tree Error')

ada_discrete_err = np.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_test)):
    ada_discrete_err[i] = zero_one_loss(y_pred, y_test)

ada_discrete_err_train = np.zeros((n_estimators,))
for i, y_pred in enumerate(ada_discrete.staged_predict(X_train)):
    ada_discrete_err_train[i] = zero_one_loss(y_pred, y_train)

ada_real_err = np.zeros((n_estimators,))
for i, y_pred in enumerate(ada_real.staged_predict(X_test)):
    ada_real_err[i] = zero_one_loss(y_pred, y_test)

ada_real_err_train = np.zeros((n_estimators,))
for i, y_pred in enumerate(ada_real.staged_predict(X_train)):
    ada_real_err_train[i] = zero_one_loss(y_pred, y_train)

dtest_error = go.Scatter(x=np.arange(n_estimators) + 1, y=ada_discrete_err,
                        mode='lines',
                        line=dict(color='red'),
                        name='Discrete AdaBoost Test Error')

dtrain_error = go.Scatter(x=np.arange(n_estimators) + 1, y=ada_discrete_err_train,
                         mode='lines',
                         line=dict(color='blue'),
                         name='Discrete AdaBoost Train Error',
                        )
rtest_error = go.Scatter(x=np.arange(n_estimators) + 1, y=ada_real_err,
                         mode='lines',
                         line=dict(color='orange'),
                         name='Real AdaBoost Test Error',
                        )
rtrain_error = go.Scatter(x=np.arange(n_estimators) + 1, y=ada_real_err_train,
                          mode='lines',
                          line=dict(color='green'),
                          name='Real AdaBoost Train Error',
                         )

layout = go.Layout(xaxis=dict(title='n_estimators'),
                   yaxis=dict(title='error rate')
                  )

data = [decision_stump, decision_tree, dtest_error, dtrain_error, rtest_error, rtrain_error]
fig = go.Figure(data=data, layout=layout)


In [5]:
py.iplot(fig)

### License

Author: 

         Peter Prettenhofer <peter.prettenhofer@gmail.com>,
         
         Noel Dawe <noel.dawe@gmail.com>

 License: 
 
         BSD 3 clause

In [2]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade
import publisher
publisher.publish(
    'Discrete versus Real AdaBoost.ipynb', 'scikit-learn/plot-adaboost-hastie-10-2/', 'Discrete versus Real AdaBoost | plotly',
    ' ',
    title = 'Discrete versus Real AdaBoost | plotly',
    name = 'Discrete versus Real AdaBoost',
    has_thumbnail='true', thumbnail='thumbnail/discrete-vs-real.jpg', 
    language='scikit-learn', page_type='example_index',
    display_as='ensemble_methods', order=15,
    ipynb= '~Diksha_Gabha/3043')

  Cloning https://github.com/plotly/publisher.git to /tmp/pip-ScmNoZ-build
  Found existing installation: publisher 0.10
    Uninstalling publisher-0.10:
      Successfully uninstalled publisher-0.10
  Running setup.py install for publisher ... - \ done
